<center>
<h1>Analisi delle vendite dei videogiochi nel mondo</h1>
<p>Antonio Pelusi</p>
<p>Matricola: 182267</p>
<p>257241@studenti.unimore.it</p>
</center>

## Indice
0. Introduzione

1. Collezione dei dati

    - Dataset vendite videogiochi

    - Dataset vendite console

2. Elaborazione dei dati

    - Pulizia records incompleti

    - Calcolo delle vendite globali

3. Analisi esplorativa
    
    - analisi_1
    
    - analisi_2

4. Risultati e conclusioni

---

## Introduzione
Nel lontano 1962, un gruppo di studenti del *Massachussetts Institute of Technology* sviluppò un gioco chiamato *Spacewar!* sul computer DEC PDP-1.

Il gioco fu presto incluso in tutti i nuovi computer DEC, diventando così il primo videogioco della storia largamente distribuito.

Da allora, il settore videoludico ha subito un processo di crescita costante, che va di pari passo con il progresso tecnologico dei computer stessi.

Il presente studio raccoglie ed analizza informazioni sulla crescita dei videogiochi dagli anni 80' (decennio della nascita delle prime console videoludiche) ad oggi in base alle copie vendute (sia continentali che globali), per poi confrontarle con la vendita di console, mettendone in risalto l'eventuale livello di correlazione.

---

## 1: Collezione dei dati
Per lo studio appena introdotto, sarà necessario avere due datasets:
- Dataset contenente le informazioni sulle vendite dei videogiochi
- Dataset contenente le informazioni sulle vendite delle console

### Dataset vendite videogiochi
Per un'analisi corretta sarà necessario avere un dataset aggiornato.
Uno dei database di riferimento nel campo videoludico è consultabile sul portale web [vgChartz.com](https://www.vgchartz.com/gamedb/).
Questo database viene costantemente aggiornato, ma non prevede il download del dataset.
Pertanto sarà necessario effettuare lo *scraping* della pagina web al fine di ottenere un dataset in un formato adeguato all'analisi (CSV, JSON ecc...).

Lo script utilizzato per lo *scraping* del sito [vgChartz.com](https://www.vgchartz.com/gamedb/) è stato sviluppato 

### Dataset vendite console


---

## 2: Elaborazione dei dati


### Pulizia records incompleti


### Calcolo delle vendite globali


---

## 3: Analisi esplorativa


### Analisi_1


---

## 4: Risultati e conclusioni
